# 3.1 Rules as Views

One way to see rules is to think of them as creating views.

All rules can be thought of in this way but some can be used to create more traditional views.

Views simplify SPARQL, reduce repetition in code, and accelerate query performance - they are incredible useful.

## Example

With the rule below, we create a view of our best performing products - that is, the products that have sold over 100 units with profit margin greater than 20%.

In [6]:
view_data = """
@prefix : <https://rdfox.com/example#> .

:productA a :Product ;
    :hasUnitsSold 1000 ;
    :hasCost 5 ;
    :hasSalePrice 10 .

:productB a :Product ;
    :hasUnitsSold 2000 ;
    :hasCost 70 ;
    :hasSalePrice 100 .

:productC a :Product ;
    :hasUnitsSold 10 ;
    :hasCost 49 ;
    :hasSalePrice 50 .

"""

In [18]:
view_rules = """
@prefix : <https://rdfox.com/example#> .

[?product, a, :highPerformingProduct],
[?product, :hasMargin, ?margin],
[?product, :hasTotalProfit, ?totalProfit] :-
    [?product, a, :Product],
    [?product, :hasUnitsSold, ?units],
    [?product, :hasCost, ?cost],
    [?product, :hasSalePrice, ?price],
    BIND ( ?price - ?cost AS ?profitPerUnit ),
    BIND ( ?profitPerUnit * ?units AS ?totalProfit ),
    BIND ( ( ?profitPerUnit / ?price ) * 100 AS ?margin ),
    FILTER (?margin > 20),
    FILTER (?units > 100).

"""

In [22]:
import requests

# Set up the SPARQL endpoint
rdfox_server = "http://localhost:12110"

# Helper function to raise exception if the REST endpoint returns an unexpected status code
def assert_response_ok(response, message):
    if not response.ok:
        raise Exception(
            message + "\nStatus received={}\n{}".format(response.status_code, response.text))

# Clear data store
clear_response = requests.delete(
    rdfox_server + "/datastores/default/content?facts=true&axioms&rules")
assert_response_ok(clear_response, "Failed to clear data store.")

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=view_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get rules
rules_response = requests.post(rdfox_server + "/datastores/default/content", data=view_rules)
assert_response_ok(rules_response, "Failed to add rule.")

# Get and issue select query
with open("../queries/3_1-RulesAsViewsQuery.rq", "r") as file:
    view_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": view_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== High Performing Products ===')
print(response.text)


=== High Performing Products ===
?product	?totalProfit	?margin	?units
<https://rdfox.com/example#productB>	60000	30.0	2000
<https://rdfox.com/example#productA>	5000	50.0	1000



## Incremental Reasoning

Adding to this, reasoning updates incrementally meaning as new data comes in or as old is removed, the view is always kept consistent with changes to the data store in real-time. This is a very efficient process which considers the smallest necessary part of the graph to compute the change.

In [24]:
new_data = """
@prefix : <https://rdfox.com/example#> .

:productD a :Product ;
    :hasUnitsSold 10000 ;
    :hasCost 10 ;
    :hasSalePrice 100 .

"""

# Add data
payload = {'operation': 'add-content-update-prefixes'}
data_response = requests.patch(
    rdfox_server + "/datastores/default/content", params=payload, data=new_data)
assert_response_ok(data_response, "Failed to add facts to data store.")

# Get and issue select query
with open("../queries/3_1-RulesAsViewsQuery.rq", "r") as file:
    view_query = file.read()
response = requests.get(
    rdfox_server + "/datastores/default/sparql", params={"query": view_query})
assert_response_ok(response, "Failed to run select query.")
print('\n=== High Performing Products ===')
print(response.text)


=== High Performing Products ===
?product	?totalProfit	?margin	?units
<https://rdfox.com/example#productD>	900000	90.0	10000
<https://rdfox.com/example#productB>	60000	30.0	2000
<https://rdfox.com/example#productA>	5000	50.0	1000



## Exercise

